In [ ]:
# import modules

import ee
from osgeo import gdal
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import datetime
from scipy import ndimage
import seaborn as sns
from mpl_toolkits.mplot3d import axes3d
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVR
from sklearn.inspection import PartialDependenceDisplay
from IPython.display import Image
from pprint import pprint
from sklearn.preprocessing import QuantileTransformer
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
# initialize the ee api through credentials
# ee.Authenticate()
ee.Initialize()

In [ ]:
# create a date list with pre-defined time-frame
dates_list = []

date_start = '2020-01-01'
date_end = '2021-01-01'
start = datetime.datetime.strptime(date_start, '%Y-%m-%d')
end = datetime.datetime.strptime(date_end, '%Y-%m-%d')
step = datetime.timedelta(days=1)
while start <= end:
    dates_list.append(start.strftime('%Y-%m-%d'))
    start += step
# print(dates_list)

In [ ]:
# load roi
fc = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level1")
roi = fc.filter(ee.Filter.eq('ADM1_NAME', 'Zuid-holland'))

In [ ]:
# load modis lst product
modis_data = ee.ImageCollection('MODIS/061/MYD11A1')

In [ ]:
# get QC values to identify days with good quality data
qc_dic = {}

for date in dates_list:
    image = modis_data.filterDate(date).first().select('QC_Day')
    qc = image.reduceRegion(reducer=ee.Reducer.median(),
                            geometry=roi.geometry(),
                            scale=1000,
                            maxPixels=1e9)

    for k, v in qc.getInfo().items():
        qc_dic[date] = v

print(qc_dic)

In [ ]:
# get dates with 0 QC i.e, good quality images
modis_dates = []
for k, v in qc_dic.items():
    if v == 0:
        modis_dates.append(k)
print(modis_dates)

In [ ]:
# filter through Sentinel-1 collection and get dates of acquisition
sentinel = ee.ImageCollection('COPERNICUS/S1_GRD')


asc = sentinel.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING')).filter(ee.Filter.eq('instrumentMode', 'IW'))
platform = asc.filter(ee.Filter.eq('platform_number', 'A'))
coll_param = platform.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')).select('VV')
sentinel_roi = coll_param.filterBounds(roi.geometry())

sar = sentinel_roi.filterDate('2020-01-01', '2021-01-01')


def date(img):
    return img.set('date', img.date().format())


sar_date = sar.map(date)

sar_dates = []
for k, v in sar_date.getInfo().items():
    for data in sar_date.getInfo()['features']:
        for k, v in data.items():
            for k, v in data['properties'].items():
                if data['properties']['date'] in sar_dates:
                    continue
                else:
                    sar_dates.append(data['properties']['date'])

print(sar_dates)

In [ ]:
# intersect the modis_dates and sar_dates to get dates for same day acquisition
for date_one in sar_dates:
    for date_two in modis_dates:
        if date_one[0:10] == date_two:
            print(date_one, date_two)

In [ ]:
# transfer images from ee to numpy array (for intersecting dates)
lst_dates = ['2020-03-25']
lst_arrs = []
modis_data = ee.ImageCollection('MODIS/061/MYD11A1')

for dates in lst_dates:
    lst = modis_data.filterDate(dates).select('LST_Day_1km').first().multiply(0.02).subtract(273.15)

    lst_new = lst.reduceResolution(reducer=ee.Reducer.median()).reproject(crs='EPSG:4326', scale=1000)

    band_arr = lst_new.sampleRectangle(region=roi.geometry(), defaultValue=-100)

    band_arr_lst = band_arr.get('LST_Day_1km')
    np_arr_lst = np.array(band_arr_lst.getInfo())
    np_arr_lst[np_arr_lst == -100] = np.nan
    lst_arrs.append(np_arr_lst)
    print(np_arr_lst.shape)

In [ ]:
# transfer images from ee to numpy array (for intersecting dates)
sentinel_dates = ['2020-03-25']
sar_arrs = []

for dates in sentinel_dates:
    sentinel = ee.ImageCollection('COPERNICUS/S1_GRD')
    asc = sentinel.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING')).filter(ee.Filter.eq('instrumentMode', 'IW'))
    platform = asc.filter(ee.Filter.eq('platform_number', 'A'))
    coll_param = platform.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')).filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')).select(['VV', 'VH'])

    sentinel_roi = coll_param.filterBounds(roi.geometry())

    bcoff = sentinel_roi.filterDate('2020-03-25', '2021-01-01').first()

    bcoff_new = bcoff.reduceResolution(reducer=ee.Reducer.median(), maxPixels=7000).reproject(crs='EPSG:4326', scale=1000)

    sar_arr = bcoff_new.sampleRectangle(region=roi.geometry(), defaultValue=-9999)

    sar_arr_VV = sar_arr.get('VV')
    sar_arr_VH = sar_arr.get('VH')

    npsar_arr_VV = np.array(sar_arr_VV.getInfo())
    sar_arrs.append(npsar_arr_VV)
    npsar_arr_VH = np.array(sar_arr_VH.getInfo())
    sar_arrs.append(npsar_arr_VH)

    print(npsar_arr_VV.shape)
    print(npsar_arr_VH.shape)

In [ ]:
# prepare arrays for plotting

lst_arr = lst_arrs[0]
print('The shape of the temperature array is:', lst_arr.shape)
print()

vv_arr = sar_arrs[0]
print('The shape of the vv_backscatter array is:', vv_arr.shape)
print()

vh_arr = sar_arrs[1]
print('The shape of the vh_backscatter array is:', vh_arr.shape)
print()

In [ ]:
figure(figsize=(14, 12), dpi=300)
plt.imshow(lst_arr, cmap='RdBu_r')
clb = plt.colorbar(orientation='horizontal')
clb.ax.set_title('\N{DEGREE SIGN}C', fontsize=12)
# plt.title('LST image over Zuid-Holland (1000 m)', y=-0.1)
plt.savefig('downscale_outputs/LST_1000.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
figure(figsize=(14, 12), dpi=300)
plt.imshow(vv_arr, cmap='Greys_r')
clb = plt.colorbar(orientation='horizontal')
clb.ax.set_title('dB', fontsize=12)
# plt.title('VV image over Zuid-Holland (1000 m)', y=-0.1)
plt.savefig('downscale_outputs/VV_1000.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
figure(figsize=(14, 12), dpi=300)
plt.imshow(vh_arr, cmap='Greys_r')
clb = plt.colorbar(orientation='horizontal')
clb.ax.set_title('dB', fontsize=12)
# plt.title('VH image over Zuid-Holland (1000 m)', y=-0.1)
plt.savefig('downscale_outputs/VH_1000.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(20, 20))
img_1 = ax[0].imshow(lst_arr, cmap='RdBu_r')
fig.colorbar(img_1, ax=ax[0], orientation='horizontal')
ax[0].set_title('LST image over Zuid-Holland (1000 m)', y=-0.5)
img_2 = ax[1].imshow(vv_arr, cmap='Greys_r')
fig.colorbar(img_2, ax=ax[1], orientation='horizontal')
ax[1].set_title('VV image over Zuid-Holland (1000 m)', y=-0.5)
img_3 = ax[2].imshow(vh_arr, cmap='Greys_r')
fig.colorbar(img_3, ax=ax[2], orientation='horizontal')
ax[2].set_title('VH image over Zuid-Holland (1000 m)', y=-0.5)
fig.show()

In [ ]:
# flatten the array to save it as a pandas dataframe
lst_flat = lst_arr.flatten()
vv_flat = vv_arr.flatten()
vh_flat = vh_arr.flatten()

In [ ]:
data_df = pd.DataFrame(lst_flat.T, columns=['lst'])
data_df['vv'] = vv_flat.T
data_df['vh'] = vh_flat.T

data_df

In [ ]:
# drop entries (rows) containing NaN data

filt_df = data_df.dropna()
filt_df

In [ ]:
filt_df.lst.hist(bins=100, legend=True)
filt_df.vv.hist(bins=100, legend=True)
filt_df.vh.hist(bins=100, legend=True)

In [ ]:
# feature scaling

scaler = QuantileTransformer()

df2 = pd.DataFrame(scaler.fit_transform(data_df),
                   columns=['lst', 'vv', 'vh'])

df2['lst'] = data_df['lst']
df2

In [ ]:
# drop entries (rows) containing NaN data

filtn_df = df2.dropna()
filtn_df

In [ ]:
print(filtn_df.lst.describe())
print()
print(filtn_df.vv.describe())
print()
print(filtn_df.vh.describe())

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(14, 10))
plt_1 = sns.regplot(data=filt_df, x='vv', y='lst', scatter_kws={'color': 'black'}, line_kws={'color': 'red'}, ci=None, ax=ax[0])
plt_1.axes.set_title('Regression Plot between VV and LST', fontsize=20)
plt_1.set_xlabel('VV', fontsize=15)
plt_1.set_ylabel('LST', fontsize=15)
plt_2 = sns.regplot(data=filt_df, x='vh', y='lst', scatter_kws={"color": "black"}, line_kws={"color": "red"}, ci=None, ax=ax[1])
plt_2.axes.set_title('Regression Plot between VH and LST', fontsize=20)
plt_2.set_xlabel('VH', fontsize=15)
plt_2.set_ylabel('LST', fontsize=15)
fig.show()
print('The correlation coefficient between VV and LST is:', filt_df['vv'].corr(filt_df['lst']))
print('The correlation coefficient between VH and LST is:', filt_df['vh'].corr(filt_df['lst']))

In [ ]:
sns.lmplot(data=filt_df, x='vv', y='lst', scatter_kws={"color": "black"}, line_kws={"color": "red"}, lowess=True)
# plt.title('LOWESS plot between VV and LST')
plt.xlabel('VV', fontsize=10)
plt.ylabel('LST', fontsize=10)
plt.savefig('downscale_outputs/LowessVV.png', dpi=300)
plt.show()
sns.lmplot(data=filt_df, x='vh', y='lst', scatter_kws={"color": "black"}, line_kws={"color": "red"}, lowess=True)
# plt.title('LOWESS plot between VH and LST')
plt.xlabel('VH', fontsize=10)
plt.ylabel('LST', fontsize=10)
plt.savefig('downscale_outputs/LowessVH.png', dpi=300)
plt.show()

In [ ]:
fig = plt.figure(figsize=(10, 8), dpi=300)
ax = plt.axes(projection='3d')
x = filt_df['vv']
y = filt_df['vh']
z = filt_df['lst']
ax.set_xlabel('VV', fontsize=8)
ax.set_ylabel('VH', fontsize=8)
ax.set_zlabel('LST', fontsize=8)
ax.set_title('3D Scatter Plot between VV, VH and LST')
ax.scatter3D(x, y, z, s=1, c='black')
plt.show()

In [ ]:
# seperate dataframe into predictors and target for model-building

predictors = filtn_df.drop(['lst'], axis=1)
target = filtn_df['lst']

print(predictors)
print()
print(target)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(predictors, target,
                                                    test_size=0.2,
                                                    random_state=11)

In [ ]:
# check what pixels are selected for training, testing and validation

train = y_train.isin(data_df['lst']).astype('int')
test = y_test.isin(data_df['lst']).astype('int')

data_df['test'] = test
data_df['train'] = train

train_px = data_df.train.values
train_px = train_px.reshape(-1, 131)

test_px = data_df.test.values
test_px = test_px.reshape(-1, 131)

figure(figsize=(20, 20), dpi=300)

plt.subplot(1, 3, 1)
plt.imshow(train_px, cmap='Greys_r')
plt.title('Training Set')

plt.subplot(1, 3, 2)
plt.imshow(test_px, cmap='Greys_r')
plt.title('Testing Set')

plt.show()

In [ ]:
# make a grid with values for hyperparameters

n_estimators = [int(x) for x in np.linspace(start=50, stop=2000, num=20)]
max_depth = [int(x) for x in np.linspace(1, 50, num=25)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}
print(random_grid)

In [ ]:
forest = RandomForestRegressor()

rf_random = RandomizedSearchCV(estimator=forest,
                               param_distributions=random_grid,
                               n_iter=100,
                               cv=5, scoring='neg_root_mean_squared_error',
                               return_train_score=True,
                               random_state=21,
                               n_jobs=-1)

rf_random.fit(x_train, y_train)

In [ ]:
print(rf_random.best_params_)

In [ ]:
base_forest = RandomForestRegressor(random_state=21)
base_forest.fit(x_train, y_train)
base_rmse = np.sqrt(mean_squared_error(base_forest.predict(x_test), y_test))
print(base_rmse)

tuned_forest = rf_random.best_estimator_
tuned_rmse = np.sqrt(mean_squared_error(tuned_forest.predict(x_test), y_test))
print(tuned_rmse)

In [ ]:
svr = SVR(kernel='linear')
svr.fit(x_train, y_train)

y_trainpred = svr.predict(x_train)
rmse_train_svr = np.sqrt(mean_squared_error(y_trainpred, y_train))
print(rmse_train_svr)

y_pred = svr.predict(x_test)
rmse_test_svr = np.sqrt(mean_squared_error(y_pred, y_test))
print(rmse_test_svr)

In [ ]:
lst_pred = tuned_forest.predict(predictors)
full_rmse = np.sqrt(mean_squared_error(lst_pred, target))
print(full_rmse)

In [ ]:
figure(figsize=(8, 8))

feature_imp = pd.Series(tuned_forest.feature_importances_,
                        index=['vv', 'vh']).sort_values(ascending=False)

sns.barplot(x=feature_imp, y=feature_imp.index)
plt.xlabel('Feature Importance Score')
plt.ylabel('Features')
plt.title("Visualizing Important Features", pad=15, size=14)

In [ ]:
figure(figsize=(20, 20), dpi=300)
common_params = { "subsample": 50, "n_jobs": 2, "grid_resolution": 40, "centered": True, "random_state": 0}
display = PartialDependenceDisplay.from_estimator(tuned_forest, predictors, features=['vv', 'vh'], kind="both", **common_params)
display.figure_.suptitle("Partial dependence")
display.figure_.subplots_adjust(hspace=0.3)

In [ ]:
gfilt_df = filtn_df.copy()
gfilt_df['lst_pred'] = lst_pred
gfilt_df

In [ ]:
print(gfilt_df['lst_pred'].describe())
print(gfilt_df['lst'].describe())

In [ ]:
gfilt_df.lst.hist(bins=150, legend=True)
gfilt_df.lst_pred.hist(bins=150, legend=True)

In [ ]:
print('The correlation between Observed LST and Predicted LST at 1000m is:',  gfilt_df['lst'].corr(gfilt_df['lst_pred']))
plt_3 = sns.regplot(data=gfilt_df, x='lst', y='lst_pred', scatter_kws={"color": "black"}, line_kws={"color": "red"}, ci=None)
plt_3.axes.set_title('Regression Plot between Observed LST and Predicted LST', fontsize=20)
plt_3.set_xlabel('Observed LST', fontsize=15)
plt_3.set_ylabel('Predicted LST', fontsize=15)
plt_3.tick_params(labelsize=5)

In [ ]:
gfilt_df = gfilt_df['lst_pred']
finaldata_df = data_df.join(gfilt_df)
finaldata_df

In [ ]:
pred_arr = finaldata_df.lst_pred.values
pred_arr = pred_arr.reshape(-1, 131)

In [ ]:
figure(figsize=(14, 12), dpi=300)
plt.imshow(pred_arr, cmap='RdBu_r')
plt.title('Predicted LST at 1000 m')
plt.colorbar(orientation='horizontal')

In [ ]:
figure(figsize=(14, 12), dpi=300)
plt.subplot(1, 2, 1)
plt.title('Predicted LST map')
plt.imshow(pred_arr[:70, :], cmap='RdBu_r')
plt.colorbar(orientation='horizontal')
plt.subplot(1, 2, 2)
plt.title('Observed LST map')
plt.imshow(lst_arr[:70, :], cmap='RdBu_r')
plt.colorbar(orientation='horizontal')
plt.show()

In [ ]:
error_arr = np.sqrt(np.square(pred_arr - lst_arr))

In [ ]:
figure(figsize=(14, 12), dpi=300)
plt.title('Error at each pixel')
plt.imshow(error_arr, cmap='RdYlGn_r')
plt.colorbar(orientation='horizontal')
plt.show()

## Downscaling 30 m (Results):

In [ ]:
# collect 30 m product

landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")

tile = landsat.filter(ee.Filter.eq('WRS_PATH', '198')).filter(ee.Filter.eq('WRS_ROW', '24'))
landsat_roi = landsat.filterBounds(roi.geometry())
landsatST = landsat_roi.select('ST_B10')

landST = landsatST.filterDate('2020-03-25', '2020-03-26').first()

task_config_l8 = {
    'maxPixels': 1e13,
    'scale': 30,
    'crs': 'EPSG:4326',
    'region': roi.geometry()
    }

landsat_task = ee.batch.Export.image(landST, 'landsatval_2503', task_config_l8)

landsat_task.start()

print('done')

In [ ]:
sentinel = ee.ImageCollection('COPERNICUS/S1_GRD')
asc = sentinel.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING')).filter(ee.Filter.eq('instrumentMode', 'IW'))
platform = asc.filter(ee.Filter.eq('platform_number', 'A'))
coll_param = platform.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')).filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')).select(['VV', 'VH'])

sentinel_roi = coll_param.filterBounds(roi.geometry())

bcoff2 = sentinel_roi.filterDate('2020-03-25', '2021-01-01').first()

task_config_s1 = {
    'maxPixels': 1e13,
    'scale': 30,
    'crs': 'EPSG:4326',
    'region': roi.geometry()
    }

sarfull_task = ee.batch.Export.image(bcoff2, 'sarval_2503', task_config_s1)

sarfull_task.start()

print('done')

In [ ]:
lst_full = gdal.Open('downscale_data/radar_assignment/landsatval_2503.tif')
lst_band = lst_full.GetRasterBand(1)
lst_full_arr = lst_band.ReadAsArray()
lst_full_arr = lst_full_arr*0.00341802+149.0-273.15
lst_full_farr = ndimage.median_filter(lst_full_arr, 3)
print(lst_full_farr.shape)

In [ ]:
# plot 30 m LST product
from matplotlib import colors

# cmap = colors.ListedColormap(['blue', 'lightblue', 'white', 'tomato', 'red', 'darkred'])
# bounds = [0, 10, 12, 15, 20, 30, 35]
# norm = colors.BoundaryNorm(bounds, cmap.N)

figure(figsize=(14, 12), dpi=300)
plt.imshow(lst_full_farr, cmap='RdBu_r')
# plt.title('LST map 30 m')
clb = plt.colorbar(orientation='horizontal')
clb.ax.set_title('\N{DEGREE SIGN}C', fontsize=12)
plt.savefig('downscale_outputs/LST_30.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
sar_full = gdal.Open('downscale_data/radar_assignment/sarval_2503.tif')
vv_band = sar_full.GetRasterBand(1)
vh_band = sar_full.GetRasterBand(2)
vv_full_arr = vv_band.ReadAsArray()
vh_full_arr = vh_band.ReadAsArray()
vv_full_farr = ndimage.median_filter(vv_full_arr, 3)
vh_full_farr = ndimage.median_filter(vh_full_arr, 3)
print(vv_full_farr.shape)
print(vh_full_farr.shape)

In [ ]:
# plot vv and vh images

figure(figsize=(14, 12), dpi=300)

# plt.title('VV image 30 m')
plt.imshow(vv_full_farr, cmap='Greys_r')
clb = plt.colorbar(orientation='horizontal')
clb.ax.set_title('dB', fontsize=12)
plt.savefig('downscale_outputs/VV_30.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
figure(figsize=(14, 12), dpi=300)
# plt.title('VH image 30 m')
plt.imshow(vh_full_farr, cmap='Greys_r')
clb = plt.colorbar(orientation='horizontal')
clb.ax.set_title('dB', fontsize=12)
plt.savefig('downscale_outputs/VH_30.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
lstfull_flat = lst_full_farr.flatten()
vvfull_flat = vv_full_farr.flatten()
vhfull_flat = vh_full_farr.flatten()

fulldata_df = pd.DataFrame(lstfull_flat.T, columns=['lst'])
fulldata_df['vv'] = vvfull_flat.T
fulldata_df['vh'] = vhfull_flat.T

fulldata_df

In [ ]:
# histograms

fulldata_df.lst.hist(bins=150, legend=True)
fulldata_df.vv.hist(bins=150, legend=True)
fulldata_df.vh.hist(bins=150, legend=True)

In [ ]:
scaler = QuantileTransformer()

scaled_df = pd.DataFrame(scaler.fit_transform(fulldata_df),
                   columns=['lst', 'vv', 'vh'])

scaled_df['lst'] = fulldata_df['lst']
scaled_df

In [ ]:
fullsar = scaled_df.drop(['lst'], axis=1)
fulllst = scaled_df.lst.values.reshape(-1, 1).ravel()

fulllst_pred = tuned_forest.predict(fullsar)

fullrmse = (np.sqrt(mean_squared_error(fulllst_pred, fulllst)))
print(fullrmse)

In [ ]:
nfulldata_df = fulldata_df.copy()
nfulldata_df['lst_pred'] = fulllst_pred
nfulldata_df

In [ ]:
fulllstpred_arr = fulllst_pred.reshape(-1, 4335)

In [ ]:
figure(figsize=(14, 12), dpi=300)
#img2 = plt.imshow(fulllstpred_arr, cmap=cmap, norm=norm)
plt.imshow(fulllstpred_arr, cmap='RdBu_r')
# plt.title('Downscaled LST map (30 m)')
clb = plt.colorbar(orientation='horizontal')
clb.ax.set_title('\N{DEGREE SIGN}C', fontsize=12)
plt.savefig('downscale_outputs/DownscaledLST_30m.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
from matplotlib import colors

cmap = colors.ListedColormap(['darkblue', 'blue', 'lightblue', 'tomato', 'red', 'darkred'])
bounds = [0, 10, 12, 15, 20, 30, 35]
norm = colors.BoundaryNorm(bounds, cmap.N)

figure(figsize=(14, 12), dpi=300)

plt.subplot(1, 2, 1)
img2 = plt.imshow(fulllstpred_arr, cmap=cmap, norm=norm)
plt.colorbar(img2, cmap=cmap, norm=norm, boundaries=bounds, ticks=bounds, orientation='horizontal')

plt.subplot(1, 2, 2)
img = plt.imshow(lst_full_farr, cmap=cmap, norm=norm)
plt.colorbar(img, cmap=cmap, norm=norm, boundaries=bounds, ticks=bounds, orientation='horizontal')

plt.show()

In [ ]:
# histogram

nfulldata_df.lst.hist(bins=200, legend=True)
nfulldata_df.lst_pred.hist(bins=200, legend=True)

In [ ]:
# error for each pixel

error_full_arr = np.sqrt(np.square(fulllstpred_arr - lst_full_farr))

figure(figsize=(14, 12), dpi=300)
#plt.title('Error at each pixel')
plt.imshow(error_full_arr, cmap='RdYlGn_r')
clb = plt.colorbar(orientation='horizontal')
clb.ax.set_title('\N{DEGREE SIGN}C', fontsize=12)
plt.savefig('downscale_outputs/erropixel_30m.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
print('The correlation between Observed LST and Downscaled LST at 30m is:', nfulldata_df['lst'].corr(nfulldata_df['lst_pred']))
plt_4 = sns.regplot(data=nfulldata_df, x='lst', y='lst_pred', scatter_kws={"color": "black"}, line_kws={"color": "red"}, ci=None)
plt_4.axes.set_title('Regression Plot between Observed LST and Downscaled LST', fontsize=20)
plt_4.set_xlabel('Observed LST', fontsize=15)
plt_4.set_ylabel('Downscaled LST', fontsize=15)